In [1]:
import numpy as np
from torch.utils.data import  DataLoader
from models.CGANGenerator import Generator
from models.CGANDiscriminator import Discriminator
from utils.audio_utils import AudioDataset
from utils.train_CGAN import train_cgan
import torch
from utils.audio_generator import generate_all_classes

In [2]:
# Load and preprocess feature and label arrays
# prepare real loader and GAN loader
raw_features = np.load("features.npy", allow_pickle=True)
features = np.stack([f.astype(np.float32) for f in raw_features])
labels = np.load("labels.npy")
audio_dataset = AudioDataset(features, labels)
Dataset = DataLoader(audio_dataset, batch_size=64, shuffle=True)

In [3]:
G = Generator(latent_dim=100, num_classes=10, output_dim=2013)
D = Discriminator(input_dim=2013, num_classes=10)

CGAN = train_cgan(generator=G, discriminator=D, dataloader=Dataset,
           num_classes=10, latent_dim=100, epochs=100, device='cuda')

Epoch [100/100]: 100%|██████████| 157/157 [00:04<00:00, 34.36it/s, D_loss=2.64e-8, G_loss=18.3]


In [ ]:
# Generate synthetic data and labels using the trained generator
# Move real features and labels to the correct device
# Concatenate real and synthetic data
# Debug shapes
synthetic_data, synthetic_labels = generate_all_classes(G, n_per_class=999)
synthetic_data = synthetic_data.view(-1, 61, 33)
device = torch.device('cuda')
real_features = torch.as_tensor(features, device=device)
real_labels = torch.as_tensor(labels, device=device)
combined_features = torch.cat([real_features, synthetic_data], dim=0)
combined_labels = torch.cat([real_labels, synthetic_labels], dim=0)
print(f"Synthetic Data Shape: {synthetic_data.shape}")
print(f"Synthetic Labels Shape: {synthetic_labels.shape}")
print(f"combined_features Shape: {combined_features.shape}")

Synthetic Data Shape: torch.Size([9990, 61, 33])
Synthetic Labels Shape: torch.Size([9990])
combined_features Shape: torch.Size([19980, 61, 33])


In [7]:
features_np = combined_features.cpu().numpy()
labels_np = combined_labels.cpu().numpy()
np.save('mfcc_CGAN_features.npy', features_np)
np.save('mfcc_CGAN_labels.npy', labels_np)